In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
def normalize_recipe_data(df):
    # Specify the required columns
    selected_columns = [
        'title', 'rating', 'calories', 'protein', 'fat', 'sodium', 'breakfast', 
        'lunch', 'dinner', 'snack', 'dessert', 'vegetarian', 'vegan', 'pescatarian',
        'paleo', 'dairy free', 'fat free', 'peanut free', 'soy free', 'wheat/gluten-free',
        'low carb', 'low cal', 'low fat', 'low sodium', 'low sugar', 'low cholesterol',
        'winter', 'spring', 'summer', 'fall', 'alcoholic', 'non-alcoholic', 'pork'
    ]

    # Select and copy the required columns
    data_filtered = df[selected_columns].copy()

    # Normalize data types
    # Convert 'title' to string
    data_filtered['title'] = data_filtered['title'].astype("string")

    # Convert float columns
    float_columns = ['rating', 'calories', 'protein', 'fat', 'sodium']
    data_filtered[float_columns] = (
        data_filtered[float_columns]
        .replace(',', '.', regex=True)
        .astype(float)
    )

    # Convert boolean columns
    boolean_columns = [col for col in selected_columns if col not in ['title'] + float_columns]
    data_filtered[boolean_columns] = data_filtered[boolean_columns].applymap(lambda x: 1 if x == 1 else 0)

    # Return the normalized DataFrame
    return data_filtered


In [3]:
csv_file = 'Data/Recipes.csv'
df = pd.read_csv(csv_file, delimiter=';')


In [4]:
normalize_df = normalize_recipe_data(df)
print(normalize_df.dtypes)

title                string[python]
rating                      float64
calories                    float64
protein                     float64
fat                         float64
sodium                      float64
breakfast                     int64
lunch                         int64
dinner                        int64
snack                         int64
dessert                       int64
vegetarian                    int64
vegan                         int64
pescatarian                   int64
paleo                         int64
dairy free                    int64
fat free                      int64
peanut free                   int64
soy free                      int64
wheat/gluten-free             int64
low carb                      int64
low cal                       int64
low fat                       int64
low sodium                    int64
low sugar                     int64
low cholesterol               int64
winter                        int64
spring                      

C:\Users\DELL\AppData\Local\Temp\ipykernel_5736\824551821.py:28: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_filtered[boolean_columns] = data_filtered[boolean_columns].applymap(lambda x: 1 if x == 1 else 0)


In [5]:
def extract_recipe_data(recipe):
    # Extract the title
    title = recipe.get("title", "")

    # Extract ingredients
    ingredients = recipe.get("ingredients", [])

    # Extract nutritional values
    calories = recipe.get("calories", 0)
    protein = recipe.get("protein", 0)
    fat = recipe.get("fat", 0)
    sodium = recipe.get("sodium", 0)

    # Extract categories
    categories = recipe.get("categories", [])

    # Extract directions
    directions = recipe.get("directions", [])

    # Combine all extracted information into a dictionary
    recipe_data = {
        "title": title,
        "ingredients": ingredients,
        "calories": calories,
        "protein": protein,
        "fat": fat,
        "sodium": sodium,
        # "categories": categories,
        "directions": directions
    }

    return recipe_data


In [6]:
def extract_recipe_data_by_title(recipes, recipe_title):
    normalized_title = recipe_title.strip().lower()

    # Filter recipes by normalized title
    matching_recipes = [recipe for recipe in recipes if "title" in recipe and recipe["title"].strip().lower() == normalized_title]
    
    if not matching_recipes:
        print(f"No matching recipe found for title: '{recipe_title}'")  # Debugging statement
        return {"len_directions": None, "len_ingredients": None}

    # Assuming we take the first matching recipe if multiple are found
    recipe = matching_recipes[0]
    
    len_directions = len(recipe.get("directions", []))
    len_ingredients = len(recipe.get("ingredients", []))
    
    return {"title": recipe["title"], "len_directions": len_directions, "len_ingredients": len_ingredients}


In [7]:
with open("Data/Recipe_Details.json", "r") as f:
    recipes_details = json.load(f)

In [8]:
search = "Lentil, Apple, and Turkey Wrap"
result = extract_recipe_data_by_title(recipes_details, search)
print(result)

{'title': 'Lentil, Apple, and Turkey Wrap ', 'len_directions': 3, 'len_ingredients': 15}


In [9]:
class NutritionNeeds:
    def __init__(self, gender, weight=None):
        """
        Initialize the nutrition needs for a specific gender and weight.

        Parameters:
        gender (str): 'male' or 'female'
        weight (float, optional): Body weight in kilograms (for protein calculation based on weight)
        """
        self.gender = gender
        self.weight = weight

        if self.gender not in ['male', 'female']:
            raise ValueError("Gender must be 'male' or 'female'.")

    def get_protein_range(self):
        """Calculate the protein intake range based on gender and weight."""
        if self.gender == 'female':
            min_protein = 46  # grams for sedentary women
            max_protein = 2 * self.weight  # for active women (up to 2g per kg body weight)
        else:  # male
            min_protein = 56  # grams for sedentary men
            max_protein = 2 * self.weight  # for active men (up to 2g per kg body weight)

        return min_protein, max_protein

    def get_calorie_range(self):
        """Calculate the calorie intake range based on gender."""
        if self.gender == 'female':
            min_calories = 1800
            max_calories = 2000
        else:  # male
            min_calories = 2200
            max_calories = 2400

        return min_calories, max_calories

    def get_fat_range(self):
        """Calculate fat intake range based on gender and total calories."""
        min_fat = 0.20 * 2000 / 9
        max_fat = 0.35 * 2000 / 9
        return min_fat, max_fat

    def get_sodium_range(self):
        """Calculate sodium intake range based on gender."""
        return 500, 2300  # milligrams

    def get_nutrition_constraints(self):
        """Return a dictionary with all nutritional constraints."""
        min_protein, max_protein = self.get_protein_range()
        min_calories, max_calories = self.get_calorie_range()
        min_fat, max_fat = self.get_fat_range()
        min_sodium, max_sodium = self.get_sodium_range()

        return {
            "calories": (min_calories, max_calories),
            "protein": (min_protein, max_protein),
            "fat": (min_fat, max_fat),
            "sodium": (min_sodium, max_sodium)
        }


In [10]:
def generate_weekly_menu(data, user_nutrition, cooldown_tracker=None, cooldown_weeks=4):
    """
    Generate a weekly menu with a cooldown system for meals.

    Parameters:
    - data (DataFrame): The meal dataset.
    - user_nutrition (NutritionNeeds): The user's nutrition constraints.
    - cooldown_tracker (dict, optional): Tracks cooldown periods for meals. Defaults to None.
    - cooldown_weeks (int): The number of weeks a meal remains on cooldown.

    Returns:
    - weekly_menu (list): A list of DataFrames, each representing a day's menu.
    - updated_cooldown_tracker (dict): Updated cooldown tracker after the week's menu.
    """
    # Initialize cooldown tracker if not provided
    if cooldown_tracker is None:
        cooldown_tracker = {}

    # Update cooldown tracker (decrement cooldowns for all meals)
    cooldown_tracker = {meal: weeks - 1 for meal, weeks in cooldown_tracker.items() if weeks > 1}

    constraints = user_nutrition.get_nutrition_constraints()
    min_calories, max_calories = constraints['calories']
    min_protein, max_protein = constraints['protein']
    min_fat, max_fat = constraints['fat']
    min_sodium, max_sodium = constraints['sodium']

    # Define meal types
    meal_types = ['breakfast', 'lunch', 'snack', 'dinner', 'dessert']
    weekly_menu = []

    # Filter out meals on cooldown
    remaining_data = data[~data['title'].isin(cooldown_tracker.keys())].copy()

    # Divide constraints by the number of meals per day
    num_meals = len(meal_types)
    min_calories /= num_meals
    max_calories /= num_meals
    min_protein /= num_meals
    max_protein /= num_meals
    min_fat /= num_meals
    max_fat /= num_meals
    min_sodium /= num_meals
    max_sodium /= num_meals

    for day in range(7):
        daily_menu = []

        for meal_type in meal_types:
            # Filter meals for the current meal type within constraints
            meal_selection = remaining_data[
                (remaining_data[meal_type] == 1) &
                (remaining_data['calories'].between(min_calories, max_calories)) &
                (remaining_data['protein'].between(min_protein, max_protein)) &
                (remaining_data['fat'].between(min_fat, max_fat)) &
                (remaining_data['sodium'].between(min_sodium, max_sodium))
            ]

            if not meal_selection.empty:
                # Select the meal closest to the middle of the constraints
                meal_selection['score'] = (
                    (meal_selection['calories'] - (min_calories + max_calories) / 2) ** 2 +
                    (meal_selection['protein'] - (min_protein + max_protein) / 2) ** 2 +
                    (meal_selection['fat'] - (min_fat + max_fat) / 2) ** 2 +
                    (meal_selection['sodium'] - (min_sodium + max_sodium) / 2) ** 2
                )
                selected_meal = meal_selection.sort_values('score').iloc[0]
                daily_menu.append(selected_meal)
                remaining_data = remaining_data.drop(selected_meal.name)  # Remove selected meal

                # Add meal to cooldown tracker
                cooldown_tracker[selected_meal['title']] = cooldown_weeks
            else:
                # Fallback: Pick any meal of the current type if no match is found
                fallback_selection = remaining_data[remaining_data[meal_type] == 1]
                if not fallback_selection.empty:
                    selected_meal = fallback_selection.iloc[0]
                    daily_menu.append(selected_meal)
                    remaining_data = remaining_data.drop(selected_meal.name)
                    cooldown_tracker[selected_meal['title']] = cooldown_weeks
                else:
                    print(f"No suitable {meal_type} found for day {day + 1}.")

        # Add daily menu to weekly menu
        if daily_menu:
            weekly_menu.append(pd.DataFrame(daily_menu))

    return weekly_menu, cooldown_tracker


In [11]:
def recommend_and_display_menu(data, user_nutrition, cooldown_tracker=None, cooldown_weeks=4):
    """
    Generate and display a weekly menu with a cooldown system for meals.

    Parameters:
    - data (DataFrame): The meal dataset.
    - user_nutrition (NutritionNeeds): The user's nutrition constraints.
    - cooldown_tracker (dict, optional): Tracks cooldown periods for meals. Defaults to None.
    - cooldown_weeks (int): The number of weeks a meal remains on cooldown.

    Returns:
    - updated_cooldown_tracker (dict): Updated cooldown tracker after the week's menu.
    """
    # Generate the weekly menu
    weekly_menu, cooldown_tracker = generate_weekly_menu(data, user_nutrition, cooldown_tracker, cooldown_weeks)

    # Display the weekly menu
    print("Weekly Menu:")
    for i, day_menu in enumerate(weekly_menu, 1):
        print(f"Day {i}:")
        print(day_menu[['title', 'calories', 'protein', 'fat', 'sodium']])
        print()  # Add spacing for readability

    return cooldown_tracker


In [12]:
# Create a NutritionNeeds object
user_nutrition = NutritionNeeds(gender='male', weight=70)

# Initialize cooldown tracker
cooldown_tracker = {}

In [13]:
cooldown_tracker = recommend_and_display_menu(normalize_df, user_nutrition, cooldown_tracker)


C:\Users\DELL\AppData\Local\Temp\ipykernel_5736\1595149478.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meal_selection['score'] = (
C:\Users\DELL\AppData\Local\Temp\ipykernel_5736\1595149478.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meal_selection['score'] = (
C:\Users\DELL\AppData\Local\Temp\ipykernel_5736\1595149478.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

Weekly Menu:
Day 1:
                                                   title  calories  protein  \
12                        Sweet Buttermilk Spoon Breads      146.0      4.0   
6      Ham Persillade with Mustard Potato Salad and M...     602.0     23.0   
1087                              Caramel Corn Blondies      271.0      8.0   
10889            Spaghetti With Tomato and Walnut Pesto      477.0     17.0   
2990             Ginger Souffles with Chocolate Sabayon      449.0     12.0   

        fat  sodium  
12      5.0   160.0  
6      41.0  1696.0  
1087   19.0    71.0  
10889  14.0   304.0  
2990   14.0   225.0  

Day 2:
                                                   title  calories  protein  \
34                         Citrus Salad with Mint Sugar      191.0      3.0   
15     Tuna, Asparagus, and New Potato Salad with Chi...     421.0     10.0   
1449             Spiced Popcorn with Pecans and Raisins      293.0      3.0   
10971      Pasta with Goat Cheese, Lemon, and Asp

In [14]:
cooldown_tracker = recommend_and_display_menu(normalize_df, user_nutrition, cooldown_tracker)

Weekly Menu:
Day 1:
                                                  title  calories  protein  \
173                         Banana Coffee Cake Diamond      273.0      4.0   
101                            Blueberry Streusel Cake      288.0      4.0   
7344                                      Banana Bread      274.0      4.0   
62            Braised Brisket with Bourbon-Peach Glaze      856.0     45.0   
49    White Chocolate Tartlets with Strawberries and...     830.0      9.0   

       fat  sodium  
173   12.0   273.0  
101   12.0   199.0  
7344  10.0   185.0  
62    54.0  1797.0  
49    59.0   148.0  

Day 2:
                                                  title  calories  protein  \
201             Breakfast Bowl With Quinoa and Berries      173.0      5.0   
111                Shrimp Cakes with Andouille Sausage      354.0     16.0   
7873                    Crispy Curry-Roasted Chickpeas      239.0     10.0   
64               Roast Chicken With Sorghum and Squash     1143.0

In [15]:
cooldown_tracker = recommend_and_display_menu(normalize_df, user_nutrition, cooldown_tracker)

No suitable snack found for day 3.
No suitable snack found for day 4.
No suitable snack found for day 5.
No suitable snack found for day 6.
No suitable snack found for day 7.
Weekly Menu:
Day 1:
                                                   title  calories  protein  \
424    Peaches and Cream Shortcakes with Cornmeal-Ora...     590.0      7.0   
195                  Cold Noodle Salad with Ponzu Sauce      200.0      8.0   
14704                    Lemon-Ginger Electrolyte Drink       99.0      3.0   
114                  Roasted Beets and Citrus with Feta      426.0      9.0   
116          Cranberry Pear Tart with Gingerbread Crust      267.0      4.0   

        fat  sodium  
424    37.0   315.0  
195     3.0   486.0  
14704   1.0   493.0  
114    28.0   408.0  
116    10.0   110.0  

Day 2:
                                                   title  calories  protein  \
439                     Cider-Glazed Mini Apple Muffins      114.0      2.0   
198                          To 